In [1]:
import nltk
import re
import csv
import unicodedata
from uszipcode import SearchEngine
import MySQLdb
import MySQLdb.cursors
import requests
import nltk
from nltk.corpus import stopwords
import string
import time

class extract_user_details():
    
    def extract_user_details_doc(self,data,pdf_to_text_list,user):#,ukNames
        
        cnx = MySQLdb.connect(host = 'localhost',
                             database = 'cv_parse',
                             user = 'root',
                             password = 'root', 
                             compress = 1,
                             cursorclass=MySQLdb.cursors.DictCursor)

        cursor = cnx.cursor()
        
        data = data
        pdf_to_text_list = pdf_to_text_list
        #print(ukNames[:5])
        #infoDict = {}
        user = user
        user['name'] = []
        user['email'] = []
        user['phone'] = []
        user['zipcode'] = []
        user['other_links'] = []
        full_name = []
        email = []
        number = []
        zipcode = []
        hit = " "
        
        #infoDict['keyword'] = []
        #print(ukNames)
        clean_txt = [word for word in data.split() if word.lower() not in stopwords.words('english')]
        nonum = [char for char in str(clean_txt) if not char.isdigit()]
        nopunc = [char for char in nonum if char not in string.punctuation] 
        data = ''.join(nopunc)
        #print(data)

        def getName():


            #for text in pdf_to_text_list:
            
            lines = [el.strip() for el in data.split() if len(el) > 0]# Split Documents from "\n" means make new lines
            lines = [nltk.word_tokenize(el) for el in lines] # TOkenize each word in each lines
            lines = [nltk.pos_tag(el) for el in lines] #
            
            print
            nameHits = []
            name = None

            grammar = r'NAME: {<NN> <NN.*> <NN.*> <NN.*>*}'
            chunkParser = nltk.RegexpParser(grammar)
            all_chunked_tokens = []
            for tagged_tokens in lines:
                # Creates a parse tree
                if len(tagged_tokens) == 0: continue # Prevent it from printing warnings
                chunked_tokens = chunkParser.parse(tagged_tokens)
                
                #print("Chunked Tokens ===",chunked_tokens)
                all_chunked_tokens.append(chunked_tokens)
                for subtree in chunked_tokens.subtrees():
                    #  or subtree.label() == 'S' include in if condition if required
                    #print("===subtree==",subtree)
                    #print(subtree.label())
                    if subtree.label() == 'NAME'or subtree.label() == 'S':
                        #p = subtree.leaves()
                        #print(p)
                        for ind, leaf in enumerate(subtree.leaves()):
                            #print(ind,"====",leaf)
                            #print("Subtress leaves if it is in given regex for Tag====",leaf[0].lower(), leaf[1])
                            #if leaf[0].lower() in ukNames:
                            #    print("check half condition====",leaf[0].lower())
                            #leaf[0] = leaf[0].replace("'","")
                            le = leaf[0]
                            #le = le.replace("'","")
                            #print("leaf ==== ",leaf[0])
                            #start_time = time.time()
                            query = """SELECT first_name FROM cv_parse.name_table
                                    WHERE '"""+ le + """' LIKE CONCAT('%',first_name,'%')"""
                            #print(query)
                            cursor.execute(query)
                            f_name = cursor.fetchall()
                            
                            #print("--- %s seconds ---" % (time.time() - start_time))
                            #print(f_name)
                            
                            if f_name and 'NN' in leaf[1]:
                            #if leaf[0].lower() in ukNames and 'NN' in leaf[1]:
                                #print("IN LOOP OF LEAF")
                                #print("subtree ====",subtree.leaves())
                                #print("leaf value===", leaf[0])
                                hit = f_name[0]['first_name']
                                #print(hit)
                                #hit = " ".join([el[0] for el in subtree.leaves()[ind:ind+3]])
                                #print("After ALL condition Name ========",hit)
                                # Check for the presence of commas, colons, digits - usually markers of non-named entities 
                                if re.compile(r'[\d,:]').search(hit): continue
                                nameHits.append(hit)
                                #print(nameHits)
                            
                                # Need to iterate through rest of the leaves because of possible mis-matches
            # Going for the first name hit
            #print("IF any name parse=====",nameHits)
            if len(nameHits) > 0:
                nameHits = [re.sub(r'[^a-zA-Z \-]', '', el).strip() for el in nameHits] 
                name = " ".join([el[0].upper()+el[1:].lower() for el in nameHits[0].split() if len(el)>0])
                otherNameHits = nameHits[1:]
                #print("Name in IF condition ====",full_name)
                full_name.append(name)
                print(full_name)
            return full_name

        def Email(text):
            index = text.find("@")
            if index > 0:
                #print(text)
                em = re.search(r'[\w\.-]+@[\w\.-]+', text)
                if em:
                    email.append(em.group(0))
                    return email

        def getPhone(text):
            pattern = re.compile(r'([+(]?\d+[)\-]?[ \t\r\f\v]*[(]?\d{2,}[()\-]?[ \t\r\f\v]*\d{2,}[()\-]?[ \t\r\f\v]*\d*[ \t\r\f\v]*\d*[ \t\r\f\v]*)')
            match = pattern.findall(text)
            if match:
                number.append(match)
            return number

        def get_zipcode(text):

            zip_code = re.search(r'([Gg][Ii][Rr] 0[Aa]{2})|((([A-Za-z][0-9]{1,2})|(([A-Za-z][A-Ha-hJ-Yj-y][0-9]{1,2})|(([A-Za-z][0-9][A-Za-z])|([A-Za-z][A-Ha-hJ-Yj-y][0-9][A-Za-z]?))))\s?[0-9][A-Za-z]{2})', text)
            if zip_code:
                zipcode.append(zip_code.group(0))
                return zipcode

        def get_links(text):
            urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)

            if len(urls) > 0:
                return urls

        na = getName()
        print("Parse Name from DOC ======",na)
        if full_name or na != None:
                user['name'] = na

        for i,text in enumerate(pdf_to_text_list):
            mail = Email(text)
            mob = getPhone(text)
            zc = get_zipcode(text)
            li = get_links(text)

            if mail != None:
                user['email'] = mail


            if mob != None:
                user['phone'] = mob


            if zc != None:
                user['zipcode'] = zc

            if li != None:
                user['other_links'] = li
                
        if len(user['email']) > 1:
            user['other_links'] = user['email'][1:]
        
        return user
    
    def extract_user_detail_pdf(self,data,pdf_to_text_list,text_prop_dict,text_dict):#,ukNames
        
        """
        This function extracts user personal data like name, address, phone, email and profile links
        from the resume

        :return: None
        """
        
        cnx = MySQLdb.connect(host = 'localhost',
                             database = 'cv_parse',
                             user = 'root',
                             password = 'root', 
                             compress = 1,
                             cursorclass=MySQLdb.cursors.DictCursor)

        cursor = cnx.cursor()

        top_text_id = -1
        max_val = -100000000

        """
        Extract Name: Iterate all the layout objects to get the text which is present
        at the top of the page which is the name of the person
        """
        # FInd closest neighbour
        def find_neighbor(key):
            """
            This function will find closest text to the given text in the resume layout
            """
            visited = []

            min_vdist, min_hdist = 1000000, 1000000
            id = 0
            neighbor = ""
            name_object = text_prop_dict.get(key)

            visited.append(name_object)
            #print (visited)
            for k, lt_obj in text_prop_dict.items():

                if (name_object.vdistance(lt_obj) + name_object.hdistance(lt_obj) <= (min_hdist + min_vdist)):

                    if (not (name_object.vdistance(lt_obj) == 0 and name_object.hdistance(lt_obj) == 0)):
                        #print(name_object.vdistance(lt_obj), name_object.hdistance(lt_obj) )
                        text = str(unicodedata.normalize('NFKD', lt_obj.get_text()).encode('utf-8'))
                        #print(text)
                        if text.__contains__("@"):
                            #print(text)
                            min_vdist = name_object.vdistance(lt_obj)
                            min_hdist = name_object.hdistance(lt_obj)
                            #print(min_vdist, min_hdist)
                            neighbor = lt_obj
                            id = k
                            #print(neighbor)
            if neighbor == "":
                return None

            visited.append(neighbor)

            return neighbor

        for key, lt_obj in text_prop_dict.items():
            if (lt_obj.bbox[1] > max_val and not lt_obj.get_text().isspace()):
                top_text_id = key
                max_val = lt_obj.bbox[1]


        "========= for Name, Email, zipcode ============"

        def getName():

            #infoDict = {}

            #for text in pdf_to_text_list:
            lines = [el.strip() for el in data.split() if len(el) > 0]# Split Documents from "\n" means make new lines
            lines = [nltk.word_tokenize(el) for el in lines] # TOkenize each word in each lines
            lines = [nltk.pos_tag(el) for el in lines] #

            otherNameHits = []
            nameHits = []
            name = None

            grammar = r'NAME: {<NN> <NN.*> <NN.*> <NN.*>*}'
            chunkParser = nltk.RegexpParser(grammar)
            all_chunked_tokens = []
            for tagged_tokens in lines:
                if len(tagged_tokens) == 0: continue # Prevent it from printing warnings
                chunked_tokens = chunkParser.parse(tagged_tokens)
                all_chunked_tokens.append(chunked_tokens)
                for subtree in chunked_tokens.subtrees():
                    #  or subtree.label() == 'S' include in if condition if required
                    #print(subtree.label())
                    if subtree.label() == 'NAME' or subtree.label() == 'S':
                        for ind, leaf in enumerate(subtree.leaves()):
                            
                            le = leaf[0]
                            le = le.replace("'","")

                            query = """SELECT first_name FROM cv_parse.name_table
                                    WHERE '"""+ le + """' LIKE CONCAT('%',first_name,'%')"""
                            
                            cnx.set_character_set('utf8')
                            cursor.execute(query)
                            f_name = cursor.fetchall()

                            if f_name and 'NN' in leaf[1]:
                            #if leaf[0].lower() in ukNames and 'NN' in leaf[1]:
                                hit = f_name[0]['first_name']
                                #hit = " ".join([el[0] for el in subtree.leaves()[ind:ind+3]])
                                #print(hit)
                                # Check for the presence of commas, colons, digits - usually markers of non-named entities 
                                if re.compile(r'[\d,:]').search(hit): continue
                                nameHits.append(hit)
                                #print(nameHits)
                                # Need to iterate through rest of the leaves because of possible mis-matches
            # Going for the first name hit
            if len(nameHits) > 0:
                nameHits = [re.sub(r'[^a-zA-Z \-]', '', el).strip() for el in nameHits] 
                name = " ".join([el[0].upper()+el[1:].lower() for el in nameHits[0].split() if len(el)>0])
                otherNameHits = nameHits[1:]


            #infoDict['name'] = name
            #infoDict['otherNameHits'] = otherNameHits

            return name

        def Email(txt):
            email = None
            pattern = re.compile(r'\S*@\S*') # Regular expressions which will find email ID from string and extract it
            matches = pattern.findall(txt) # Gets all email addresses as a list
            return matches

        def getPhone(txt):
            number = None
            try:
                #pattern = re.compile(r'([+(]?\d+[)\-]?[ \t\r\f\v]*[(]?\d{2,}[()\-]?[ \t\r\f\v]*\d{2,}[()\-]?[ \t\r\f\v]*\d*[ \t\r\f\v]*\d*[ \t\r\f\v]*)')
                pattern = re.compile(r'\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}')
                match = pattern.findall(txt)
                # match = [re.sub(r'\s', '', el) for el in match]
                    # Get rid of random whitespaces - helps with getting rid of 6 digits or fewer (e.g. pin codes) strings
                # substitute the characters we don't want just for the purpose of checking
                match = [re.sub(r'[,.]', '', el) for el in match if len(re.sub(r'[()\-.,\s+]', '', el))>6]
                    # Taking care of years, eg. 2001-2004 etc.
                match = [re.sub(r'\D$', '', el).strip() for el in match]
                    # $ matches end of string. This takes care of random trailing non-digit characters. \D is non-digit characters
                match = [el for el in match if len(re.sub(r'\D','',el)) <= 15]
                    # Remove number strings that are greater than 15 digits
                try:
                    for el in list(match):
                        # Create a copy of the list since you're iterating over it
                        if len(el.split('-')) > 3: continue # Year format YYYY-MM-DD
                        for x in el.split("-"):
                            try:
                                # Error catching is necessary because of possibility of stray non-number characters
                                # if int(re.sub(r'\D', '', x.strip())) in range(1900, 2100):
                                if x.strip()[-4:].isdigit():
                                    if int(x.strip()[-4:]) in range(1900, 2100):
                                        # Don't combine the two if statements to avoid a type conversion error
                                        match.remove(el)
                            except:
                                pass
                except:
                    pass
                number = match
            except:
                pass
            return number

        "=========== Functions if not parse from original function"

        #print(top_text_id)
        def get_name(top_text_id):
            """
            Extract name from the text
            """

            email_flag = False
            phone_flag = False
            link_flag = False

            name_object = text_prop_dict.get(top_text_id)
            text = name_object.get_text()

            full_name = []

            lines = [el.strip() for el in text.split() if len(el) > 0]# Split Documents from "\n" means make new lines
            lines = [nltk.word_tokenize(el) for el in lines] # TOkenize each word in each lines
            lines = [nltk.pos_tag(el) for el in lines] #

            otherNameHits = []
            nameHits = []
            name = None

            grammar = r'NAME: {<NN.*> <NN.*> <NN.*>*}'
            #print(grammar)
            # Noun phrase chunk is made out of two or three tags of type NN. (ie NN, NNP etc.) - typical of a name. {2,3} won't work, hence the syntax
            # Note the correction to the rule. Change has been made later.
            chunkParser = nltk.RegexpParser(grammar)
            all_chunked_tokens = []
            for tagged_tokens in lines:
                # Creates a parse tree
                if len(tagged_tokens) == 0: continue # Prevent it from printing warnings
                chunked_tokens = chunkParser.parse(tagged_tokens)
                all_chunked_tokens.append(chunked_tokens)
                for subtree in chunked_tokens.subtrees():
                    #  or subtree.label() == 'S' include in if condition if required
                    #print(subtree.label())
                    if subtree.label() == 'NAME' or subtree.label() == 'S':
                        p = subtree.leaves()
                        #print(p)
                        for ind, leaf in enumerate(subtree.leaves()):
                            #print(ind,"====",leaf)
                            #print(leaf[0].lower(), leaf[1])
                            le = leaf[0]
                            le = le.replace("'","")
                            query = """SELECT first_name FROM cv_parse.name_table
                                    WHERE '"""+ le + """' LIKE CONCAT('%',first_name,'%')"""
                            
                            cnx.set_character_set('utf8')
                            cursor.execute(query)
                            f_name = cursor.fetchall()

                            if f_name and 'NN' in leaf[1]:
                            #if leaf[0].lower() in ukNames and 'NN' in leaf[1]:
                                hit = f_name[0]['first_name']
                                #hit = " ".join([el[0] for el in subtree.leaves()[ind:ind+3]])
                                #print(hit)
                                # Check for the presence of commas, colons, digits - usually markers of non-named entities 
                                if re.compile(r'[\d,:]').search(hit): continue
                                nameHits.append(hit)
                                #print(nameHits)
                                # Need to iterate through rest of the leaves because of possible mis-matches
            # Going for the first name hit
            if len(nameHits) > 0:
                nameHits = [re.sub(r'[^a-zA-Z \-]', '', el).strip() for el in nameHits] 
                name = " ".join([el[0].upper()+el[1:].lower() for el in nameHits[0].split() if len(el)>0])
                otherNameHits = nameHits[1:]

            full_name.append(name)
            #print(full_name)

            # check email in name object
            index = text.find("@")
            if index > 0:
                email_flag = True

            # check phone number in name object
            cell = re.search("\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}", text)
            if cell:
                phone_flag = True
            #name = " ".join(full_name)

            # check hyperlinks in name object
            #print(text)
            urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
            if len(urls) > 0:
                link_flag = True

            return name, email_flag, phone_flag, link_flag

        def get_email(text):
            """
            Search email using regex from the given text
            :param text: Text which consist of email address
            :return: None
            """
            #print(text)

            text = str(unicodedata.normalize('NFKD', text).encode('utf-8'))
            index = text.find("@")
            if index > 0:
                email = re.search(r'[\w\.-]+@[\w\.-]+', text)
                if email:
                    return email.group(0)
            else:
                for k,v in text_dict.items():
                    v = str(unicodedata.normalize('NFKD', v).encode('utf-8'))
                    text = re.search(r'[\w\.-]+@[\w\.-]+', v)
                    if text:
                        return text.group(0)
            return None

        def get_cell(text):

            text = str(unicodedata.normalize('NFKD', text).encode('utf-8'))
            cell = re.search("\d{5}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}", text)
            if cell:
                return cell.group(0)
            else:
                for k,v in text_dict.items():
                    v = str(unicodedata.normalize('NFKD', v).encode('utf-8'))
                    cell = re.search("\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}", v)

                    if cell:
                        return cell.group(0)
                return None

        def get_zipcode(text):

            text = str(unicodedata.normalize('NFKD', text).encode('utf-8'))

            if text:
                zip_code = re.search(r'([Gg][Ii][Rr] 0[Aa]{2})|((([A-Za-z][0-9]{1,2})|(([A-Za-z][A-Ha-hJ-Yj-y][0-9]{1,2})|(([A-Za-z][0-9][A-Za-z])|([A-Za-z][A-Ha-hJ-Yj-y][0-9][A-Za-z]?))))\s?[0-9][A-Za-z]{2})',text)
                if zip_code:
                    return(zip_code.group(0))
                return None
            zip_code = re.search(r'\b\d{5}(?:[-\s]\d{4})?\b', text)
            #print (zip_code)
            if zip_code:
                return zip_code.group(0)
            else:
                for k,v in text_dict.items():
                    v = str(unicodedata.normalize('NFKD', v).encode('utf-8'))
                    zip_code = re.search(r'\b\d{5}(?:[-\s]\d{4})?\b', v)
                    if zip_code:
                        return zip_code.group(0)
            return None

        def get_links(text):

            text = str(unicodedata.normalize('NFKD', text).encode('utf-8'))
            urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)

            if len(urls) > 0:
                return urls
            else:
                result = []
                for k, v in text_dict.items():
                    v = str(unicodedata.normalize('NFKD', v).encode('utf-8'))
                    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',v)
                    if len(urls) > 0:
                        result.extend(urls)
            return result

        user = {}
        user['name'] = []
        user['email'] = []
        user['phone'] = []
        user['zipcode'] = []
        #user['other_links'] = []

        neighbor = find_neighbor(top_text_id)
        #print(neighbor)

        full_name, email_flag, phone_flag, link_flag = get_name(top_text_id)
        if full_name:
            user['name'].append(full_name)
        else:
            user['name'].append(None)

        if not email_flag and not neighbor == None:
            user['email'].append((get_email(neighbor.get_text())))
        else:
            name_object = text_prop_dict.get(top_text_id)
            text = name_object.get_text()
            user['email'].append(get_email(text))
            if not user['email']:
                user['email'].append(Email(data))

        if not phone_flag and not neighbor == None:
            user['phone'].append(get_cell(neighbor.get_text()))
        else:
            name_object = text_prop_dict.get(top_text_id)
            text = name_object.get_text()
            user['phone'].append((get_cell(text)))
            if not user['phone']:
                user['phone'].append((getPhone(data)))

        if not neighbor == None:

            zipcode = get_zipcode(neighbor.get_text())

            # Use zipcode dictionary to find state and city
            #search = SearchEngine()
            #zip_obj = search.by_zipcode(zipcode)
            #state = zip_obj.state
            #city = zip_obj.major_city
            user['zipcode'].append(zipcode)
        else:
            zipcode = get_zipcode(data)
            user['zipcode'].append(zipcode) 

        if not link_flag and not neighbor == None:
            user['other_links'] = (get_links(neighbor.get_text()))
        else:
            name_object = text_prop_dict.get(top_text_id)
            text = name_object.get_text()
            user['other_links'] = (get_links(text))

        return user

/Users/imac086/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
ud = extract_user_details()

In [4]:
ud.extract_user_details_doc()

TypeError: extract_user_details_doc() missing 4 required positional arguments: 'data', 'pdf_to_text_list', 'ukNames', and 'user'

In [36]:
import re
s= 'ihj'
if re.search(r'^[a-zA-Z]{1}$',s) is None:
    print("More than one character")
    print

More than one character
